In [ ]:
# Global data variables
SANDBOX_NAME = ''# Sandbox Name
DATA_PATH = "/data/sandboxes/" + SANDBOX_NAME + "/data/data/" 




# Organizar datos

Una de las fases del proceso de data wrangling consiste en dar una estructura a los datos, normálmente esta fase conlleva las siguientes operaciones:

- Establecer índices, renombrar columnas.

- Ordenar valores.

- Eliminar duplicados

- Filtrar registros y/o columnas

- Editar información

- Modificar la estructura de los datos



<div class="alert alert-danger" role="alert">
  <strong>NOTA:</strong> Los métodos para organizar los datos de DataFrames de Spark son siempre <b>transformaciones</b>. Es importante recordar que el resultado de una transformación de un DataFrame es siempre otro DataFrame.
</div>

In [ ]:
vancouver_df = spark.read.csv(DATA_PATH + 'crime_in_vancouver.csv', sep=',', header=True, inferSchema=True)
pokemon_df = spark.read.csv(DATA_PATH + 'pokemon.csv', sep=',', header=True, inferSchema=True)

In [ ]:
pokemon_df.show(6)

In [ ]:
vancouver_df.show(5)



## Filtrar Columnas

### Select

El método `select` es una **transformación** para seleccionar un subconjunto de columnas. `select` puede recibir una lista de columnas o los nombres de las columnas como parámetros independientes. Funciona como la sentencia SQL _SELECT_.

In [ ]:
pk_name_type = pokemon_df.select('Name', 'Type 1', 'Type 2')
pk_name_type.show(5)

In [ ]:
columns = ['NEIGHBOURHOOD', 'TYPE', 'YEAR']

vancouver_df.select(columns).show(5)



Extrae las 3 primeras columnas del dataframe de Vancouver

In [ ]:
vancouver_df.select(vancouver_df.columns[:3]).show()



Extrae las columnas que empiezan por M del dataframe de vancouver

In [ ]:
vancouver_df.select([x for x in vancouver_df.columns if x.startswith('M')]).show()



### Drop

El método `drop` tiene la función contraria al `select`, elimina un subconjunto de columnas. En este caso no se puede pasar una lista de columnas, es necesario utlizar el operador `*` para convertirlo a parámetros indivuales.

**OJO:** Si se intenta eliminar una columna que no existe no devuelve error.

In [ ]:
vancouver_df = vancouver_df.drop('X', 'Y', 'Z')
vancouver_df.show(3)

In [ ]:
drop_columns = ['Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']

pokemon_df.drop(*drop_columns).show(3)



## Renombrar columnas


### Una columna

El método para renombrar columnas en _pyspark_ es `withColumnRenamed`. Este método recibe dos parámetros, el nombre de la columna original y la nueva, por tanto sirve para renombar una única columna.

In [1]:
pokemon_df.withColumnRenamed('Type 1', 'type_1').show(3)

NameError: ignored

 

Recuerda que se pueden concatenar transformaciones.

In [2]:
vancouver_df.withColumnRenamed('YEAR', 'year').withColumnRenamed('DAY', 'day').show(3)

NameError: ignored



### Varias columnas

Por ejemplo, imaginemos que queremos convertir a mínusculas y sin espacios todos los nombres de las columnas. En spark hay dos opciones: un bucle con `withColumnRenamed` o utilizar el método `select` con `alias`.

Renombra todas las columnas de pokemon df con `withColumnRenamed`, poniéndolas en minúsculas, sustituyendo espacios por barra baja, y eliminando puntos de los títulos:

In [3]:
for col in pokemon_df.columns:
    pokemon_df = pokemon_df.withColumnRenamed(col, col.lower().replace(' ', '_').replace('.', ''))

NameError: ignored

In [4]:
pokemon_df.show(3)

NameError: ignored



El módulo `pyspark.sql.functions` contiene todas las funciones de spark implementadas para tratar con DataFrames. La función `alias` se puede utilizar junto con `select` para seleccionar una columna de un DataFrame cambiándole el nombre.

Usando `select` + `alias`:

In [5]:
from pyspark.sql import functions as F

ModuleNotFoundError: ignored

In [ ]:
F.col('TYPE').alias('type')



Cambia a minúsculas las columnas TYPE y NEIGHBOURHOOD en la misma línea de código.

In [ ]:
vancouver_df.select(F.col('TYPE').alias('type'), F.col('NEIGHBOURHOOD').alias('neighbourhood')).show(3)



Aplica esa misma transformación en todas las columnas (minúsculas, y espacios por barra baja)

In [ ]:
[F.col(c).alias(c.lower().replace(' ', '_')) for c in vancouver_df.columns]

In [ ]:
vancouver_df = vancouver_df.select([F.col(c).alias(c.lower().replace(' ', '_')) for c in vancouver_df.columns])

In [ ]:
vancouver_df.show(3)



## Filtrar Registros

Los métodos `filter` y `where` se utlizan para quedarse con registros que cumplan cierta condición. Se pueden utlizar indistintamente. Para poner la condición es necesario usar la función `F.col()` para indicar a _spark_ el nombre de la columna del filtro.



__Valor exacto__

Si es un número entero; extrae los crímenes que se hayan producido en el 2008:

In [ ]:
vancouver_2008 = vancouver_df.filter(F.col('year') == 2008)
vancouver_2008.show(5)

In [ ]:
vancouver_2008.count()

In [ ]:
vancouver_df.count()



Si es un string; extrae las filas del pokemon Pikachu

In [ ]:
pokemon_df.where(F.col('name') == 'Pikachu').show(10)

 

__Mayor/Menor que__

Extrae todos los crímenes que se han realizado más tarde del 2008. ¿Cuántos se han producido?

In [ ]:
vancouver_more_2008 = vancouver_df.filter(F.col('year') >= 2008)
vancouver_more_2008.show(5)

In [ ]:
vancouver_more_2008.count()



__Contiene substring__

Encuentra aquellos crímenes que contengan en la columna `type` el substring `heft`

Hint: Utiliza `like()`

In [ ]:
vancouver_df.filter(F.col('type').like("%heft%")).show(5)



__Valor en/no en lista__

1. Extrae los crímenes que se cometieron en West End, Kitsilano o Killarney.
2. Extrae los crímenes que no se cometieron ni en 2005 ni en 2003.

Hint: Usa el operando `~` para indicar negación, `isin()` para incluir las condiciones.

In [ ]:
neighbourhoods = ['West End', 'Kitsilano', 'Killarney']

vancouver_df.where(F.col('neighbourhood').isin(neighbourhoods)).show(5)

In [ ]:
vancouver_df.filter(~F.col('year').isin([2005, 2003])).show(5)



__Combinación de filtros (AND / OR)__

Muestra los homicidios que se cometieron en 2007.

In [ ]:
vancouver_df.where((F.col('type') == 'Homicide') & (F.col('year') == 2007)).show(3)



Muestra los crímenes que se cometieron más tarde del 2006 o que sean robos de vehículo (`Theft of Vehicle`).

In [ ]:
vancouver_df.filter((F.col('year') >= 2006) | (F.col('type') == 'Theft of Vehicle')).show(3)

 

## Registros Duplicados

Una de las fases del data wrangling es la identificación y eliminación de registros duplicados. 

__dropDulicates__

`dropDuplicates` toma un subconjunto de columnas para identificar duplicados y devuelve un nuevo DataFrame sin los registros duplicados.

In [ ]:
pokemon_df.count()

In [ ]:
pokemon_nodup = pokemon_df.dropDuplicates()
pokemon_nodup.show(3)

In [ ]:
pokemon_nodup.count()

 

No hay duplicados teniendo en cuenta todas las columnas. Considerando únicamente las columnas *type_1* y *type_2* hay varios.

In [ ]:
pokemon_nodup.dropDuplicates(subset=['type_1', 'type_2']).count()



__distinct__

Una llamada al método `distinct` es lo mismo que al método `dropDuplicates` sin parámetro. Es decir, tiene en cuenta todas las columnas. También se utiliza normalmente para contar los valores únicos de una columna.

In [ ]:
pokemon_df.distinct().count()

In [ ]:
pokemon_df.select('type_1').distinct().count()



Cuenta cuántos valores distintos hay en la columna `year` del dataset de crimen en Vancouver, y muestra cuáles son

In [ ]:
vancouver_df.select('year').distinct().show()

In [ ]:
vancouver_df.select('year').distinct().count()



## Ordenar DataFrames

Ambos métodos `sort` y `orderBy` pueden ser usados indistintamente para ordenar DataFrames. Se utilizan los métodos `asc` y `desc` sobre las columnas para indicar si el orden es ascendiente o descendiente. Se puede ordenar por múltiples columnas.

In [ ]:
pokemon_df.orderBy(F.col('attack').desc()).show(10)

In [ ]:
pokemon_df.sort(F.col('attack').asc()).show(5)

In [ ]:
pokemon_df.sort(F.col('attack').asc(), F.col('hp').desc()).show(5)



## Agrupar registros

`groupBy` sirve para agrupar los datos sobre los campos indicados haciendo una operación sobre ellos. Las operaciones a calcular se indican dentro de `agg()` y deben encontrarse dentro de `pyspark.sql.functions`. 

In [ ]:
pokemon_df.groupBy('type_1').agg(F.avg('attack')).show(5)



Se puede utilizar `alias` para definir el nombre de la columna de salida.

In [ ]:
pokemon_df.groupBy('type_1').agg(F.avg('defense').alias('avg_def')).show(5)



Se pueden incluir en el groupBy multiples columnas y operaciones.

In [ ]:
pokemon_df.groupBy('type_1', 'legendary').agg(F.max('hp')).show(5)



Muestra en base al valor de `legendary` cuál es la media de `attack`, `defense` y `hp`

In [ ]:
pokemon_df.groupBy('legendary').agg(F.avg('attack'), F.avg('defense'), F.avg('hp')).show()



El caso excepcional es si la operación es un `count` que no necesita el método `agg`.

Cuenta cuántos crímenes se han producido en cada año.

In [ ]:
vancouver_df.groupBy('year').count().show()



Recuerda que se pueden contactenar transformaciones

In [ ]:
vancouver_df.groupBy('year').count().orderBy(F.col('year').desc()).show()



## Limitar el número de registros

El método `limit` devuelve un nuevo DataFrame con únicamente _N_ filas.

In [ ]:
vancouver_df.count()

In [ ]:
vancouver_1000 = vancouver_df.limit(1000)
vancouver_1000.show(5)

In [ ]:
vancouver_1000.count()



## Ejercicio 1

Dado el fichero

In [ ]:
vancouver_df = spark.read.csv(DATA_PATH + '/crime_in_vancouver.csv', sep=',', header=True, inferSchema=True)



- a. Cargue el fichero en un dataframe y estudie las estructura de los datos
- b. Transforme los nombres de las columnas a minúsculas y con los espacios convertidos a "_" para seguir las buenas prácticas de nombrado de variables
- c. Elimine la columna 'y'
- d. Elimine las filas duplicadas (teniendo en cuenta todas las columnas)
- e. ¿Cuántos crímenes se cometieron en 2003?
- f. ¿En qué barrio se han cometido más crímenes?

In [ ]:
# Respuesta aqui

In [ ]:
# Respuesta aqui

In [ ]:
# Respuesta aqui

In [ ]:
# Respuesta aqui

In [ ]:
# Respuesta aqui

In [ ]:
# Respuesta aqui